In [15]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
import os
from langgraph.types import Command
from langgraph.prebuilt import create_react_agent
from langchain.tools import tool
from typing_extensions import Literal
from langgraph.graph import MessagesState, StateGraph, START, END

In [16]:
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

llm = ChatOpenAI()
llm.invoke("Hey there, how are you?")

AIMessage(content="Hello! I'm just a virtual assistant, so I don't have feelings, but I'm here to help you. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 14, 'total_tokens': 45, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Bnt4QZfRJYA7dNXOFmog4sPSNCDBD', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--23e1b8e0-3781-4ea9-8200-c25e7dc07da9-0', usage_metadata={'input_tokens': 14, 'output_tokens': 31, 'total_tokens': 45, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [17]:
def add_number(state):
    result = state["num1"] + state["num2"]
    print(f"addition is {result}")
    
    return Command(goto = "multiply", update = {"sum": result})

In [18]:
state = {"num1": 5, "num2": 10}
add_number(state)

addition is 15


Command(update={'sum': 15}, goto='multiply')

In [19]:
@tool
def transfer_to_addition_expert():
    """Ask addition agent for help"""
    return

@tool
def transfer_to_multiplication_expert():
    """Ask multiplication agent for help"""
    return

In [20]:
llm_with_tool = llm.bind_tools([transfer_to_multiplication_expert, transfer_to_addition_expert])

In [21]:
response = llm_with_tool.invoke("What is 5 + 10?")
response.content

''

In [22]:
response.tool_calls

[{'name': 'transfer_to_addition_expert',
  'args': {},
  'id': 'call_b4IltmQfM5PY5wZAimecaEpQ',
  'type': 'tool_call'}]

In [23]:
system_prompt = ("You are an addition expert, you can ask the multiplication expert for help with multiplication"
                 "Always do your portion of calculation first, then ask the multiplication expert if needed")

messages = [{"role": "system", "content": system_prompt}] + ["Can you add 5 and 10?"]

In [24]:
messages

[{'role': 'system',
  'content': 'You are an addition expert, you can ask the multiplication expert for help with multiplicationAlways do your portion of calculation first, then ask the multiplication expert if needed'},
 'Can you add 5 and 10?']

In [31]:
def addition_agent(state: MessagesState) -> Command[Literal["multiplication_agent", "__end__"]]:
    system_prompt = "You are an addition expert, and you can ask for help from the multiplication expert when asked to multiply numbers.Always do your portion of calculation first, and then ask the multiplication expert if needed."
    messages = [{"role": "system", "content": system_prompt}] + state["messages"]
    
    llm = llm.bind_tools([transfer_to_multiplication_expert])
    ai_message = llm.invoke(messages)
    
    if len(ai_message.tool_calls) > 0:
        tool_call_id = ai_message.tool_calls[-1].id
        tool_message = {"role": "tool", "content": "Successfully transferred to multiplication expert.", "tool_call_id": tool_call_id}

        return Command(goto="multiplication_agent", update={"messages": messages + [ai_message, tool_message]})

    return {"messages":[ai_message]}


def multiplication_agent(state: MessagesState) -> Command[Literal["addition_agent", "__end__"]]:
    system_prompt = "You are a multiplication expert, and you can ask for help from the addition expert when asked to add numbers.Always do your portion of calculation first, and then ask the addition expert if needed."
    messages = [{"role": "system", "content": system_prompt}] + state["messages"]
    
    llm = llm.bind_tools([transfer_to_addition_expert]).invoke(messages)
    ai_message = llm.invoke(messages)
    
    if len(ai_message.tool_calls) > 0:
        tool_call_id = ai_message.tool_calls[-1].id
        tool_message = {"role": "tool", "content": "Successfully transferred to addition expert.", "tool_call_id": tool_call_id}

        return Command(goto="addition_agent", update={"messages": messages + [ai_message, tool_message]})

    return {"messages":[ai_message]}

In [32]:
graph = StateGraph(MessagesState)

graph.add_node("addition_agent", addition_agent)
graph.add_node("multiplication_agent", multiplication_agent)

graph.add_edge(START, "addition_agent")

app = graph.compile()

In [33]:
app.invoke({"messages": [("user", "Can you add 5 and 10?")]})

UnboundLocalError: cannot access local variable 'llm' where it is not associated with a value